In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA

# Read data

In [ ]:
train_f = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_ts = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_tns = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
test_f = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

train_f.shape, train_ts.shape, train_tns.shape, test_f.shape, sub.shape

# g

In [ ]:
df_g = train_f[[col for col in train_f.columns if col.startswith('g-')]]

pca_g = PCA(n_components=0.85)
df_g_pca = pd.DataFrame(pca_g.fit_transform(df_g))

del df_g

# c

In [ ]:
df_c = train_f[[col for col in train_f.columns if col.startswith('c-')]]

pca_c = PCA(n_components=0.9)
df_c_pca = pd.DataFrame(pca_c.fit_transform(df_c))

del df_c

# final df

In [ ]:
train_f_final = pd.concat([train_f.iloc[:,:4],df_g_pca,df_c_pca],axis=1,ignore_index=True)

del df_g_pca
del df_c_pca

mask = train_f_final[1]=='trt_cp'

X_train = train_f_final[mask].iloc[:,2:]
y_train = train_ts[mask].iloc[:,1:]

X_train[3]=X_train[3].map({'D1':1,'D2':2})

# Model

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost.sklearn import XGBClassifier

In [ ]:
clf_xgb = OneVsRestClassifier(XGBClassifier(tree_method='gpu_hist',n_jobs=-1,
                                            colsample_bytree=0.6746045225305067,
 gamma= 8.717282170474643,
 learning_rate= 0.0676566413896137,
 max_depth= 15,
 min_child_weight= 10,
 n_estimators=199,
 subsample=0.8772601635382853
                                                  ),n_jobs=-1)


clf_xgb.fit(X_train,y_train)


# Pre-process test

g

In [ ]:
df_g = test_f[[col for col in test_f.columns if col.startswith('g-')]]

df_g_pca = pd.DataFrame(pca_g.transform(df_g))

del df_g

c

In [ ]:
df_c = test_f[[col for col in test_f.columns if col.startswith('c-')]]

df_c_pca = pd.DataFrame(pca_c.transform(df_c))

del df_c

final df

In [ ]:
test_f_final = pd.concat([test_f.iloc[:,:4],df_g_pca,df_c_pca],axis=1,ignore_index=True)

del df_g_pca
del df_c_pca

X_test = test_f_final.iloc[:,2:]

X_test[3]=X_test[3].map({'D1':1,'D2':2})

# Predict

In [ ]:
test_pred = clf_xgb.predict_proba(X_test)

mask_test = test_f['cp_type']=='ctl_vehicle'
test_pred[mask_test] = 0

# submission

In [ ]:
submission = pd.DataFrame(test_f['sig_id'])
submission = pd.concat([submission,pd.DataFrame(test_pred)],axis=1)
submission.columns = train_ts.columns
submission.to_csv('submission.csv', index=False)